In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('HumanNet-PI.tsv', sep='\t')
network = []
for i in range(len(df)):
    network.append(df.index[i])
# print(network)
net = pd.DataFrame(network)

In [4]:
import networkx as nx

G = nx.from_pandas_edgelist(net, source=0, target=1, create_using=nx.Graph())
valid_nodes = list(G.nodes)

def calculate_network_proximity1(G, node1, node2, k ):
    try:
        shortest_path = nx.shortest_path_length(G, source=node1, target=node2)
        network_proximity = 1 / shortest_path
        return network_proximity
    except:
        return 0
    
def calculate_network_proximity2(G, node1, node2 , k):
    try:
        shortest_path = nx.approximate_current_flow_betweenness(G, [(node1, node2)], k=k)
        network_proximity = 1 / shortest_path
        return network_proximity
    except:
        return 0

In [5]:
drug_df = pd.read_csv('processed_drug_simmat.csv', index_col=0)
disease_df = pd.read_csv('processed_disease_simmat.csv', index_col=0)

In [6]:
import json

with open('gene_dict_drug.json', 'r') as f:
    gene_dict_drug = json.load(f)
with open('gene_dict_disease.json', 'r') as f:
    gene_dict_disease = json.load(f)

In [7]:
drug_names = drug_df.index
disease_names = disease_df.index

In [8]:
new_gene_dict_drug = {}
for key, value in gene_dict_drug.items():
    if(key in drug_names):
        value_int_list = [int(x) for x in value]
        intersection = list(set(value_int_list).intersection(valid_nodes))
        new_gene_dict_drug[key] = intersection

In [9]:
new_gene_dict_disease = {}
for key, value in gene_dict_disease.items():
    if(key in disease_names):
        value_int_list = [int(x) for x in value]
        intersection = list(set(value_int_list).intersection(valid_nodes))
        new_gene_dict_disease[key] = intersection

In [10]:
from tqdm import tqdm

In [15]:
labels = pd.read_csv('labelssmall.csv', index_col=0)

In [16]:
labels

0        1  2
0     D013920  DB00396  0
1     D017449  DB00401  0
2     D013285  DB01399  0
3     D009157  DB00178  0
4     D003324  DB00876  0
...       ...      ... ..
4995  D019970  DB00714  1
4996  D058186  DB00364  1
4997  D009101  DB01069  1
4998  D006937  DB00722  1
4999  D011471  DB00443  1

[10000 rows x 3 columns]

In [ ]:
proximities = []
for index, row in tqdm(labels.iterrows()):
    drug_genes = new_gene_dict_drug[row[1]]
    disease_genes = new_gene_dict_disease[row[0]]

    network_proximities = []
    for drug_gene in drug_genes:
        for disease_gene in disease_genes:
            network_proximities.append(calculate_network_proximity2(G, drug_gene, disease_gene,20))


    mean = sum(network_proximities) / len(network_proximities)
    median = sorted(network_proximities)[len(network_proximities) // 2]
    minimum = min(network_proximities)
    maximum = max(network_proximities)                                   
    proximities.append((mean, median, minimum, maximum))   

6282it [05:32,  6.54it/s]

In [ ]:
import numpy as np
dd_list = []
label_list = []
dds_list = []
for index, row in labels.iterrows():
    drug_name = row[1]
    disease_name = row[0]
    label = row[2]
    drug_vector = list(drug_df.loc[drug_name,:].values)
    disease_vector = list(disease_df.loc[disease_name,:].values)
    proximity_vector = list(proximities[index])
    
    label_list.append(label)
    dd_vector = drug_vector + disease_vector + [label]
    dds_vector = dd_vector + proximity_vector + [label]
    dd_vector = np.array(dd_vector)
    dds_vector = np.array(dds_vector)
    dd_list.append(dd_vector)
    dds_list.append(dds_vector)
    

In [ ]:
import numpy as np

# Save the list of arrays
np.save('dds10000_20.npy', dds_list)
np.save('dd10000_20.npy', dd_list)

In [ ]:
# Load the list of arrays
loaded_arr_list = np.load('dds10000.npy', allow_pickle=True)

# Print the loaded list of arrays
print(len(loaded_arr_list))

In [ ]:
print((loaded_arr_list[10].shape))

In [ ]:
# Load the list of arrays
loaded_arr_list = np.load('dd10000.npy', allow_pickle=True)

# Print the loaded list of arrays
print(len(loaded_arr_list))

print((loaded_arr_list[10].shape))

In [ ]:
loaded_arr_list[10:20][-6:]

In [ ]:
proximities